This notebook trains and tests an SCA agent on the Laser Hockey Enviroment

In [8]:
# Imports
import numpy as np
import laserhockey.hockey_env as h_env
from importlib import reload
from SAC import Agent
import matplotlib.pyplot as plt
# from gym import wrappers

In [9]:
# Global variables
train_games = 200
test_games = 100
change_mode_after = 100
load_checkpoint = True
train = True
# render testing
render = False

In [10]:
# Initialization 
np.set_printoptions(suppress=True)
reload(h_env)

# get enviroment
env = h_env.HockeyEnv(mode=h_env.HockeyEnv.TRAIN_DEFENSE)

#init agent
agent = Agent(input_dims=env.observation_space.shape, 
              env=env,
              n_actions=int(env.action_space.shape[0]/2))

# and oponent
opponent = h_env.BasicOpponent()


c:\Users\Emil\anaconda3\envs\RL\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
c:\Users\Emil\anaconda3\envs\RL\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [11]:
# Plotting function
def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)

In [12]:
# Training of agent

# uncomment this line and do a mkdir tmp && mkdir video if you want to
# record video of the agent playing the game.
#env = wrappers.Monitor(env, 'tmp/video', video_callable=lambda episode_id: True, force=True)

best_score = env.reward_range[0]
score_history = []

if train:
    # conttinue from checkpoint
    if load_checkpoint:
        agent.load_models()

    for i in range(train_games):
        observation, info = env.reset()
        obs_opponent = env.obs_agent_two()

        done = False
        score = 0

        while not done:
            action1 = agent.act(observation)
            action2 = opponent.act(obs_opponent)
            
            observation_, reward, done, _, info = env.step(np.hstack([action1,action2]))
            score += reward
            agent.remember(observation, action1, reward, observation_, done)
            
            # learn after buffer is filled 
            if i > 20:
                agent.learn()
                
            obs_opponent = env.obs_agent_two()
            observation = observation_
            
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score and i > 100:
            best_score = avg_score
            agent.save_models()

        print('episode ', i, 'score %.1f' % score, 'avg_score %.1f' % avg_score)

    x = [i+1 for i in range(train_games)]
    plot_learning_curve(x, score_history, 'SCA.png')

.... loading models ....
episode  0 score -6.2 avg_score -6.2
episode  1 score -1.7 avg_score -3.9
episode  2 score -10.6 avg_score -6.2
episode  3 score -3.7 avg_score -5.6
episode  4 score -4.3 avg_score -5.3
episode  5 score 0.0 avg_score -4.4
episode  6 score -1.7 avg_score -4.0
episode  7 score -2.1 avg_score -3.8
episode  8 score -2.1 avg_score -3.6
episode  9 score -2.0 avg_score -3.4
episode  10 score -1.4 avg_score -3.3
episode  11 score -2.8 avg_score -3.2
episode  12 score -1.7 avg_score -3.1
episode  13 score -2.9 avg_score -3.1
episode  14 score -1.8 avg_score -3.0
episode  15 score -1.2 avg_score -2.9
episode  16 score -4.0 avg_score -3.0
episode  17 score -1.6 avg_score -2.9
episode  18 score -3.7 avg_score -2.9
episode  19 score -11.7 avg_score -3.4
episode  20 score -5.2 avg_score -3.4
episode  21 score -2.4 avg_score -3.4
episode  22 score -1.2 avg_score -3.3
episode  23 score -2.0 avg_score -3.3
episode  24 score -2.7 avg_score -3.2
episode  25 score -2.3 avg_score -

In [ ]:
# Testing

wins = 0
losses = 0
draws = 0

agent.load_models()
opponent = h_env.BasicOpponent()

for i in range(test_games):
    if render:
        env.render() 
    observation, info = env.reset()
    obs_opponent = env.obs_agent_two()
    done = False
    while not done:
        if render:
            env.render() 
        action1 = agent.act(observation)
        action2 = opponent.act(obs_opponent)

        observation, reward, done, _, info = env.step(np.hstack([action1,action2]))

        obs_opponent = env.obs_agent_two()
    if info['winner']==1:
            wins+=1
    elif info['winner']==-1:
            losses+=1
    else:
        draws+=1
        
wins, losses, draws

.... loading models ....


c:\Users\Emil\RL-Hockey\SAC.py:201: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  state = T.Tensor([observation]).to(self.actor.device)


8